## Create a notebook to perform Runge-Kutta integration for multiple coupled varibles

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

## Define our coupled derivatives to integrate

In [ ]:
def dydx(x,y):
    
    #set the derivatives
    
    #our equation is d^2y/dx^2 = -y
    
    #so we write 
    #dydx = z
    #dzdx = -y
    
    #we will set y = y[0]
    #we will set z = y[1]
    
    #declare an array
    y_derivs = np.zeros(2)
    
    #set dydx = z
    y_derivs[0] =  y[1]
    
    #set dzdx = -y
    y_derivs[1] = -1*y[0]
    
    #here we have to return an array
    return y_derivs
    

## Coefficient Matricies

In [ ]:
c_arr = np.array([0.0,1.0/5.0,3.0/10.0,3.0/5.0,1.0,7.0/8.0])

a_arr = np.array([[0.0,0.0,0.0,0.0,0.0,0.0],
     [1.0/5.0,0.0,0.0,0.0,0.0,0.0],
     [3.0/40.0,9.0/40.0,0.0,0.0,0.0,0.0],
     [3.0/10.0,-9.0/10.0,6.0/5.0,0.0,0.0,0.0],
     [-11.0/54.0,5.0/2.0,-70.0/27.0,35.0/27.0,0.0,0.0],
     [1631.0/55296.0,175.0/512.0,575.0/13824.0,44275.0/110592.0,253.0/4096.0,0.0]])

b_arr = np.array([37.0/378.0,0.0,250.0/621.0,125.0/594.0,0.0,512.0/1771.0])

bstar_arr = np.array([2825.0/27648.0,0.0,18575.0/48384.0,13525.0/55296.0,277.0/14336.0,1.0/4.0])


## Define the Cash-Karp method

In [ ]:
def ck_mv_core(dydx,xi,yi,nv,h):
    
    #declare 6 by nv array
    k = np.zeros((6,nv))
    
    #declare an x array
    x_coef = np.zeros(6)
    x_coef = xi + c*h
                 
    #declare a temp y array
    y_temp = np.zeros(nv)
    
    #get k1 values
    y_derivs = dydx(xi,yi)
    k[0] = h*y_derivs[:]
    
    #variable to iterate
    i=1
    
    #get k2 through k6 values
    while i <= 5:
        
        y_temp[:] = yi[:] + np.dot(a_arr[i],k[:])
        y_derivs = dydx(x_coef[i],y_temp)
        k[i][:] = h*y_derivs[:]
        
        i+=1
    
    #advance y by a step h
    yipo = yi + np.dot(b_arr,k)
    yipostar = yi + np.dot(bstar_arr,k)
    
    return yipo, yipostar

## Define an adaptive step size driver for CK

In [ ]:
def ck_mv_ad(dydx,x_i,y_i,nv,h,tol):
    
    #define safety scale
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    
    #set a meximum number of iterations
    imax = 10000
    
    #set an iteration variable
    i = 0
    
    #create an error
    Delta = np.full(nv,2*tol)
    
    #remember the step
    h_step = h
    
    #adjust step
    while(Delta.max()/tol > 1.0):
        
        #estimate our error by subtracting y from ystar
        y_ck = ck_mv_core(dydx,x_i,y_i,nv,h_step)
        
        #compute an error
        Delta = np.fabs(y_ck[0]-y_ck[1])
        
        #if the error is too large, take a smaller step
        if(Delta.max()/tol > 1.0):
            
            #our error is too large, decrease the step
            h_step *= SAFETY*(Delta.max()/tol)**(-0.25)
            
        #check iteration
        if(1>=imax):
            print("Too many iterations in rk_mv_ad()")
            raise StopIteration("Ending after i = ",i)
            
        #iterate
        i+=1
        
    
    #next time try to take a bigger step
    h_new = np.fmin(h_step * (Delta.max()/tol + tol/1e4)**(-0.9), h_step*H_NEW_FAC)
    
    #return the answer, a new step, and the step we actually took
    return y_ck[0], h_new, h_step
 

## Define a wrapper for Cash-Karp

In [ ]:
def ck_mv(dfdx,a,b,y_a,tol):
    
    #dfdx is the derivative wrt x
    #a is the lower bound
    #b is the upper bound
    #y_a are the boundary conditions
    #tol is the tolerance for integrating y
    
    #define our starting step
    xi = a 
    yi = y_a.copy()
    
    #an initial step size == make very small!
    h = 1.0e-4 * (b-a)
    
    #set maximum number of iterations
    imax = 10000
    
    #set an iteration variable
    i = 0
    
    #set teh number of coupled odes to the
    #size of y_a
    nv = len(y_a)
    
    #set the initial conditions
    x = np.full(1,a)
    y = np.full((1,nv),y_a)
    
    #set a flag
    flag = 1
    
    #loop until we reach the right side
    while(flag):
        
        #calculate y_i+1
        yi_new, h_new, h_step = ck_mv_ad(dydx,xi,yi,nv,h,tol)
        
        #update the step
        h = h_new
        
        #prevent an overshoot
        if(xi+h_step>b):
            
            #take a smaller step
            h = b-xi
            
            #recalculate y_i+1
            yi_new, h_new, h_step = ck_mv_ad(dydx,xi,yi,nv,h,tol)
            
            #break 
            flag = 0
        
        #update values
        xi += h_step
        yi[:] = yi_new[:]
    
        #add the step to the arrays
        x = np.append(x,xi)
        y_new = np.zeros((len(x),nv))
        y_new[0:len(x)-1,:] = y
        y_new[-1,:] = yi[:]
        del y
        y = y_new
    
        #prevent too many iterations
        if(i>=imax):
        
            print("Maximum iterations reached.")
            raise StopIteration("Iteation number = ", i)
        
        #iterate
        i += 1 
    
        #output some information
        s = "i= %3d\tx = %9.8f\th = %9.8f\tb=%9.8f" % (i,xi,h_step, b)
        print(s)
    
        #break if new xi is == b
        if(xi==b):
            flag = 0
        
    #return the answer
    return x,y

## Perform the integration

In [ ]:
a = 0.0
b = 2.0 * np.pi

y_0 = np.zeros(2)
y_0[0] = 0.0
y_0[1] = 1.0
nv = 2

tolerance = 1.0e-6

#perfrom the integration 
x,y = ck_mv(dydx,a,b,y_0,tolerance)

## Plot the result

In [ ]:
plt.plot(x,y[:,0],'o',label='y(x)')
plt.plot(x,y[:,1],'o',label='dydx(x)')
xx = np.linspace(0,2.0*np.pi,1000)
plt.plot(xx,np.sin(xx),label='sin(x)')
plt.plot(xx,np.cos(xx),label='cos(x)')
plt.xlabel('x')
plt.ylabel('y, dy/dx')
plt.legend(frameon=False)


err_y = y[:,0] - np.sin(x)
err_dydx = y[:,1] - np.cos(x)

## Plot the error

In [ ]:
err_y = y[:,0] - np.sin(x)
err_dydx = y[:,1] - np.cos(x)

plt.plot(x,err_y, label='y(x) error')
plt.plot(x,err_dydx, label='dydx(x) error')

plt.xlabel('x')
plt.ylabel('y(x) error, dydx(x) error')
plt.legend(frameon=False)